In [1]:
!pip install -q kaggle

!mkdir -p ~/.kaggle

In [2]:
from google.colab import files
uploaded = files.upload()

!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle (1).json


In [3]:
!kaggle competitions download -c aith-dl-competition-text-data-2025

  0% 0.00/13.1M [00:00<?, ?B/s]
100% 13.1M/13.1M [00:00<00:00, 512MB/s]


In [4]:
import zipfile
import os

file_path='/content/aith-dl-competition-text-data-2025.zip'
with zipfile.ZipFile(file_path, 'r') as archive:
      target_dir = os.path.splitext(file_path)[0]

      os.makedirs(target_dir, exist_ok=True)
      archive.extractall(target_dir)

In [5]:
import pandas as pd
import os

data_path = '/content/aith-dl-competition-text-data-2025'
train_data = pd.read_csv(os.path.join(data_path, 'train.csv'))
test_data = pd.read_csv(os.path.join(data_path, 'test.csv'))

In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173803 entries, 0 to 173802
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      173803 non-null  int64 
 1   text    173803 non-null  object
 2   score   173803 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.0+ MB


In [7]:
train_data.head(20)

,id,text,score
0,127857,пидорасы наше правительство издеваються над пе...,1
1,86622,чё цена?,0
2,33294,старый пиндосовский прихвостень...,1
3,421,хорошо потрудились... молодец!!!,0
4,118550,хаба это скучная возня по полу👎,0
5,109253,пидор ряженый.,1
6,140411,"женщины,это развод,не подписывайтесь. 2 месяца...",0
7,115129,"разорвать суку, на куски, нечесть ебанная",1
8,241938,"❗ заказ отправлен, ждём счёт ❗",0
9,173581,можно сколько цена,0


In [23]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    train_data,
    test_size=0.2,
    random_state=42,
    stratify=train_data['score']  # для сбалансированного разделения
)
test_data_path = "/content/aith-dl-competition-text-data-2025/test.csv"  # Укажите правильный путь
test_df = pd.read_csv(test_data_path)

In [24]:
import nltk
from nltk.tokenize import word_tokenize
import html
import re

nltk.download('punkt_tab')


nltk.download('punkt')
nltk.download('punkt_tab')

def remove_punctuation_nltk(text):

    tokens = word_tokenize(text)

    filtered_tokens = []
    for token in tokens:
        if token.isalnum():
            filtered_tokens.append(token)

    cleaned_text = ' '.join(filtered_tokens)
    return cleaned_text



def clean_text(text):
  text = text.lower()

  text = html.unescape(text)

  text = re.sub(r'https?://\S+|www\.\S+', '', text)

  text = re.sub(r'@\w+', '', text)

  text = re.sub(r'#(\w+)', r'\1', text)

  text = remove_punctuation_nltk(text)

  return text


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [25]:
train_df["cleaned_text"] = train_df["text"].apply(clean_text)
val_df["cleaned_text"] = val_df["text"].apply(clean_text)
test_df["cleaned_text"] = test_df["text"].apply(clean_text)


In [29]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from transformers import AutoTokenizer

class TextDataset(Dataset):
    """Кастомный Dataset для текстовой классификации"""

    def __init__(self, dataframe, tokenizer, max_length=128, has_labels=True):
        """
        Args:
            dataframe: pandas DataFrame с колонками 'text' и 'label'
            tokenizer: токенайзер из transformers
            max_length: максимальная длина последовательности
            has_labels: есть ли в данных метки (True для train/val, False для test)
        """
        self.dataframe = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.has_labels = has_labels

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = str(self.dataframe.iloc[idx]['cleaned_text'])

        item_id = None
        if 'id' in self.dataframe.columns:
            item_id = self.dataframe.iloc[idx]['id']

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        item = {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

        if item_id is not None:
            item['id'] = item_id

        if self.has_labels and 'score' in self.dataframe.columns:
            label = self.dataframe.iloc[idx]['score']
            if label is not None:
                item['labels'] = torch.tensor(label, dtype=torch.float)
        elif self.has_labels and 'label' in self.dataframe.columns:
            label = self.dataframe.iloc[idx]['label']
            if label is not None:
                item['labels'] = torch.tensor(label, dtype=torch.long)

        return item



In [33]:
import pandas as pd
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

train_dataset = TextDataset(train_df, tokenizer, max_length=128)
val_dataset = TextDataset(val_df, tokenizer, max_length=128)
test_dataset = TextDataset(
    test_df,
    tokenizer,
    max_length=128,
    has_labels=False
)

batch_size = 32

train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=True,
    drop_last=True
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


In [35]:
from transformers import AutoModelForSequenceClassification
from tqdm import tqdm


model_name = "sismetanin/rubert-toxic-pikabu-2ch"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [34]:

model.eval()

predictions = []
ids = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Processing batches"):
        if 'id' in batch:
            ids.extend(batch['id'].tolist())
        else:
            batch_size_current = len(batch['input_ids'])
            start_idx = len(ids)
            ids.extend(range(start_idx, start_idx + batch_size_current))

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1)

        for i in range(len(probs)):
            neg_score = probs[i][0].item()
            pos_score = probs[i][1].item()

            if pos_score > neg_score:  # POSITIVE
                toxicity_score = max(0.0, (1 - pos_score) * 0.3)
            else:  # NEGATIVE
                toxicity_score = min(neg_score * 1.2, 1.0)

            predictions.append(toxicity_score)

print(f"Total predictions: {len(predictions)}")
print(f"Total IDs: {len(ids)}")

if len(ids) == len(predictions) and not all(isinstance(i, int) for i in ids):
    final_ids = ids
else:
    final_ids = test_df['id'].tolist()[:len(predictions)]
    print(f"Using IDs from DataFrame (first {len(predictions)})")

submission_df = pd.DataFrame({
    'id': final_ids,
    'score': predictions
})

submission_path = 'submission.csv'
submission_df.to_csv(submission_path, index=False)

Using device: cuda


Processing batches: 100%|██████████| 2328/2328 [07:44<00:00,  5.01it/s]


Total predictions: 74487
Total IDs: 74487
Using IDs from DataFrame (first 74487)
